# Capstone Project: selamat pagi anak-anakku

- Amelia Yunisa
- Mochamad Bayu Safutra
- Rio Ferdinand
- Rizka Salisa Puteri

## Import Library

In [3]:
import os, glob, random, shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import tensorflow as tf
# import tensorflowjs as tfjs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback 

## Load Dataset Resep

In [5]:
df = pd.read_csv('dataset_resep.csv')

print(df.head())

print(df.info())

# print(df.describe())

print(df.isnull().sum())
print(df.duplicated().sum())

              Nama                                          Deskripsi  \
0      Ayam Goreng  Ayam Goreng adalah hidangan tradisional Indone...   
1        Ayama Pop  Ayam Pop adalah hidangan tradisional Indonesia...   
2            Bakso  Bakso adalah hidangan tradisional Indonesia ya...   
3  Dendeng Batokok  Dendeng merupakan daging kering yang diiris ti...   
4             Gado  Gado-gado adalah hidangan tradisional Indonesi...   

                                               Resep  
0  Bahan-bahan:\n1 kg Ayam, cuci bersih dan poton...  
1  Bahan-bahan :\n1 ekor ayam kampung (bisa pake ...  
2  Bahan-bahan\n250 gr Fillet ayam bagian paha (s...  
3  Bahan-bahan:\n1 kg daging sapi\n1 ons Cabai me...  
4  Bahan-bahan:\n1 buah timun (iris tipis)\n2 tan...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nama       32 non-null     object
 1   Deskrip

## Splitting Data

In [8]:
import os, glob, random, shutil

src_root  = 'food'
dst_root  = 'food_split'
splits    = {'train': .7, 'validation': .15, 'test': .15}

# 1) Ambil daftar kelas dari src_root/train
classes = os.listdir(os.path.join(src_root, 'train'))

# 2) Buat direktori tujuan per split & per kelas
for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(dst_root, split, cls), exist_ok=True)

# 3) Gabungkan file per kelas dari train/validation/test
for cls in classes:
    all_files = []
    for sub in ['train','validation','test']:
        pattern = os.path.join(src_root, sub, cls, '*')
        all_files += glob.glob(pattern)

    random.seed(42)
    random.shuffle(all_files)
    n = len(all_files)
    n_train = int(splits['train'] * n)
    n_val   = int(splits['validation']   * n)

    train_files = all_files[:n_train]
    val_files   = all_files[n_train:n_train+n_val]
    test_files  = all_files[n_train+n_val:]

    # 4) Copy ke folder baru
    for f in train_files:
        shutil.copy(f, os.path.join(dst_root, 'train', cls))
    for f in val_files:
        shutil.copy(f, os.path.join(dst_root, 'validation', cls))
    for f in test_files:
        shutil.copy(f, os.path.join(dst_root, 'test', cls))

    print(f"{cls}: train={len(train_files)}, validation={len(val_files)}, test={len(test_files)}")


ayam_goreng: train=102, validation=22, test=23
ayam_pop: train=147, validation=31, test=33
bakso: train=1507, validation=323, test=324
dendeng_batokok: train=145, validation=31, test=32
gado: train=1501, validation=321, test=323
grontol: train=210, validation=45, test=45
gudeg: train=328, validation=70, test=71
gulai_ikan: train=153, validation=32, test=34
gulai_tambusu: train=126, validation=27, test=28
gulai_tunjang: train=154, validation=33, test=34
kue_ape: train=169, validation=36, test=37
kue_bika_ambon: train=142, validation=30, test=32
kue_cenil: train=142, validation=30, test=31
kue_dadar_gulung: train=162, validation=34, test=36
kue_gethuk_lindri: train=154, validation=33, test=33
kue_kastengel: train=154, validation=33, test=33
kue_klepon: train=168, validation=36, test=36
kue_lapis: train=168, validation=36, test=37
kue_lemper: train=202, validation=43, test=44
kue_lumpur: train=173, validation=37, test=38
kue_nagasari: train=161, validation=34, test=35
kue_pastel: train=16

## Load Dataset

In [9]:
# 1. Atur path dan parameter global
base_dir   = os.getcwd()
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

## Augmentasi Data

In [13]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,         # Lebih kecil
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

#    b) Hanya rescale untuk validation dan test
val_test_gen = ImageDataGenerator(rescale=1./255)

# 3. Buat generator
train_generator = train_gen.flow_from_directory(
    os.path.join(dst_root, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    shuffle=True
)

validation_generator = val_test_gen.flow_from_directory(
    os.path.join(dst_root, 'validation'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_gen.flow_from_directory(
    os.path.join(dst_root, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 9175 images belonging to 32 classes.
Found 1982 images belonging to 32 classes.
Found 1999 images belonging to 32 classes.


## Modeling

In [14]:
# 1. Load Pretrained MobileNetV2
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# 2. Bangun model transfer learning kamu
model = Sequential([
    base_model,

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')  # jumlah kelas dari generator
])

# 3. Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4. Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=3, factor=0.2, monitor='val_loss')

# 5. Fit model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stop, reduce_lr]
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 128)      │     1,474,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,900,896 (14.88 MB)

 Trainable params: 1,642,912 (6.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 200s 686ms/step - accuracy: 0.4322 - loss: 2.2183 - val_accuracy: 0.7225 - val_loss: 0.9876 - learning_rate: 0.0010
Epoch 2/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 185s 643ms/step - accuracy: 0.7181 - loss: 0.9905 - val_accuracy: 0.7815 - val_loss: 0.7656 - learning_rate: 0.0010
Epoch 3/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 139s 484ms/step - accuracy: 0.7905 - loss: 0.7235 - val_accuracy: 0.8073 - val_loss: 0.7028 - learning_rate: 0.0010
Epoch 4/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 124s 433ms/step - accuracy: 0.8289 - loss: 0.5977 - val_accuracy: 0.8355 - val_loss: 0.6343 - learning_rate: 0.0010
Epoch 5/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 133s 464ms/step - accuracy: 0.8554 - loss: 0.5140 - val_accuracy: 0.8385 - val_loss: 0.6074 - learning_rate: 0.0010
Epoch 6/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 126s 440ms/step - accuracy: 0.8657 - loss: 0.4814 - val_accuracy: 0.8179 - val_loss: 0.7392 - learning_rate: 0.0010
Epoch 7/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 126s 439ms/step - accura

In [18]:
saved_model_dir = os.path.join(base_dir, 'saved_model_tl')
model.export(saved_model_dir)
print(f"SavedModel folder created at: {saved_model_dir}")

INFO:tensorflow:Assets written to: c:\Users\Ferdinand\Downloads\Capstone\saved_model_tl\assets


INFO:tensorflow:Assets written to: c:\Users\Ferdinand\Downloads\Capstone\saved_model_tl\assets


Saved artifact at 'c:\Users\Ferdinand\Downloads\Capstone\saved_model_tl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_482')
Output Type:
  TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)
Captures:
  2825789676656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824281103472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2825848952384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824281105232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824281108224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824281111392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824288226640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824288225056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824288226464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2824288224880: TensorSpec(shape=(), dtype=tf.resource, name=N

## Evaluation

## Inference